In [1]:
# import sys

# sys.path.insert(0, '/content/drive/MyDrive/RecSys/FinalProject/src/')

In [31]:
# import argparse
# import pickle
# import lib
# from lib.utils import read_data
# from pathlib import Path
# import os 
# from tqdm import tqdm
# from collections import Counter
# import numpy as np
# import torch
# import pandas as pd 

# from IPython.display import display

Digenetica

In [46]:
data_folder = '/content/drive/MyDrive/RecSys/FinalProject/data/digenetica'

You choose

In [40]:
data_folder = '/content/drive/MyDrive/RecSys/FinalProject/data/youchoose'

In [47]:
train_data = 'train'
valid_data = 'valid'

print("Loading train data from {}".format(os.path.join(data_folder, train_data)))
print("Loading valid data from {}".format(os.path.join(data_folder, valid_data)))
data_path = Path(data_folder)
train_file_name = train_data
val_file_name = valid_data
train_data, valid_data = read_data(data_path, train_file_name, val_file_name)
display(train_data.df.head())

Loading train data from /content/drive/MyDrive/RecSys/FinalProject/data/digenetica/train
Loading valid data from /content/drive/MyDrive/RecSys/FinalProject/data/digenetica/valid


,SessionID,UserID,ItemID,Time,EventDate,item_idx
128,1,NaN,9654,1.464215e+09,2016-05-09,2
523,1,NaN,33043,1.464215e+09,2016-05-09,3
61,1,NaN,32118,1.464215e+09,2016-05-09,1
716,1,NaN,12352,1.464215e+09,2016-05-09,4
1039,1,NaN,35077,1.464215e+09,2016-05-09,5


In [49]:
data  = train_data.df.set_index('item_idx').append(valid_data.df.set_index('item_idx'))
data['item_count'] = data.groupby('ItemID').transform('count').iloc[:, 0]
data.head()

,SessionID,UserID,ItemID,Time,EventDate,item_count
item_idx,,,,,,
2,1,NaN,9654,1.464215e+09,2016-05-09,395
3,1,NaN,33043,1.464215e+09,2016-05-09,194
1,1,NaN,32118,1.464215e+09,2016-05-09,67
4,1,NaN,12352,1.464215e+09,2016-05-09,326
5,1,NaN,35077,1.464215e+09,2016-05-09,102


In [50]:
item2count = data['item_count'].sort_values(ascending=False).reset_index().drop_duplicates().set_index('item_idx')
data = valid_data.df.set_index('item_idx')
data['item_count'] = data.groupby('ItemID').transform('count').iloc[:, 0]
n_sessions = data['SessionID'].nunique()

In [51]:
K = 20
mrrs = []
recalls = []

# For each sessions
for n, g in tqdm(data.groupby('SessionID'), total=n_sessions):
  session_pop = Counter()
  session = []

  for i, target in enumerate(g.index):
    recommendations = []
    session_pop = Counter(session)
    session.append(target)
    
    if i == 0:
      continue

    occour = session_pop.most_common(K)
    items = list(map(lambda t: t[0], occour))
    local_counts = list(map(lambda t: t[1], occour))
    global_counts = list(map(lambda it: item2count.loc[it].iloc[0], items))

    df = pd.DataFrame([items, local_counts, global_counts], index=['item', 'local', 'global']).T
    
    df.sort_values(by=['local', 'global'], inplace=True, ascending=False)
    recommendations = list(df['item'])
    n_rec = len(recommendations)

    if n_rec < K:
      recommendations += list(item2count.iloc[: K-n_rec].index)
    elif n_rec >= K:
      recommendations = recommendations[:K]

    target = torch.tensor(target)
    recommendations = torch.tensor(recommendations).view(1, K)
    recall = lib.metric.get_recall(recommendations, target)
    mrr = lib.metric.get_mrr(recommendations, target)

    recalls.append(recall)
    mrrs.append(mrr)

mean_recall = np.mean(recalls)
mrrs = [t.item() for t in mrrs]
mean_mrr = np.mean(mrrs)

print()
print(mean_recall, mean_mrr)

100%|██████████| 2175/2175 [00:23<00:00, 91.43it/s] 


0.22701983551040156 0.1459944194599979


Another popularity model idea

In [ ]:
B = 1
dataloader = lib.DataLoader(valid_data, B)

In [ ]:
K = 20
mrrs = []
recalls = []
cache = {}

for ii, (input, target, _) in tqdm(enumerate(dataloader), total=len(dataloader.dataset.df)):
  input = input.item()

  if not input in cache:
    item_data = data.loc[input]
    # display(item_data.set_index('SessionID').loc[8891])

    c = Counter()

    if len(item_data.shape) == 1:
      sessions = [item_data['SessionID']]
    else:
      sessions = item_data['SessionID'].to_numpy()
    
    for s in sessions:
      session_items = list(data[data['SessionID'] == s].index)
      # session_items = list(g.get_group(s).index)
      _ =  c.update(session_items)
    
    c.pop(input)
    most_common_count = c.most_common(K)
    outputs =  torch.tensor(list(map(lambda t: t[0], most_common_count))).long().view(B, -1)

    recall = lib.metric.get_recall(outputs, target)
    mrr = lib.metric.get_mrr(outputs, target)
    cache[input] = (recall, mrr)

  recalls.append(cache[input][0])
  mrrs.append(cache[input][1])


mean_recall = np.mean(recalls)
# Added by Yarden
mrrs = [t.item() for t in mrrs]
mean_mrr = np.mean(mrrs)

 79%|███████▉  | 8268/10443 [07:36<02:00, 18.11it/s]


In [ ]:
mean_mrr

0.1241820516104413

In [ ]:
mean_recall

0.3935655539429124

In [ ]:
for ii, (input, target, _) in enumerate(dataloader):
  input = input.item()
  item_data = data.loc[input]
  # display(item_data.set_index('SessionID').loc[8891])

  c = Counter()
  i = 0

  if ii == 1414:
    break

In [ ]:
len(item_data.shape)

1